In [3]:
import numpy as np
from pulp import *
from scipy.optimize import minimize
import pandas as pd
np.set_printoptions(edgeitems=100, linewidth=100000)


In [307]:
prob = LpProblem('Healthy_Pet_Food', LpMaximize)
M = LpVariable('M', 0)
Y = LpVariable('Y', 0)
prob += 0.65*M + 0.45*Y, 'z'

prob += 2*M + 3*Y <= 400000, 'cereal constraint'
prob += 3*M + 1.5*Y <= 300000, 'meat constraint'
prob += M <= 90000, 'meaties capacity'

prob.solve()

print('Status', LpStatus[prob.status])
print('Objective', prob.objective.value())
for v in prob.variables():
    print(v.varValue)
pd.DataFrame([{'var':name, 'shadow price':c.pi, 'slack':c.slack}
            for name, c in prob.constraints.items()])

Status Optimal
Objective 77500.0
50000.0
100000.0


,var,shadow price,slack
0,cereal_constraint,0.0625,-0.0
1,meat_constraint,0.1750,-0.0
2,meaties_capacity,-0.0000,40000.0


In [286]:
for name, c in prob.constraints.items():
    print(name, c.pi, c.slack)

cereal_constraint 0.0625 -0.0
meat_constraint 0.175 -0.0
meaties_capacity -0.0 40000.0


In [196]:
model = LpProblem('Resource-allocation', LpMaximize)
x = {i: LpVariable(f"x{i}", 0) for i in range(1, 5)}
y = {i: LpVariable(f"y{i}", cat=LpBinary) for i in (1, 3)}
model += 20*x[1] + 12*x[2] + 40*x[3] + 25*x[4], 'profit'

M = 100
model += lpSum(x.values()) <= 50, 'manpower'
model += 3*x[1] + 2*x[2] + x[3] <= 100, 'material A'
model += x[2] + 2*x[3] + 3*x[4] <= 90, 'material B'
model += x[1] <= y[1] * M, 'x1 constraint'
model += x[3] <= y[3] * M, 'x3 constraint'
model += y[1] + y[3] <= 1, 'y constraint'

model.solve()

print('Objective', model.objective.value())
print('Status', LpStatus[model.status])
for v in model.variables():
    print(v, '=', v.varValue)

o = [{'name':name, 'shadow price':c.pi, 'slack':c.slack}
    for name, c in model.constraints.items()]
pd.DataFrame(o)

Objective 1800.0
Status Optimal
x1 = 0.0
x2 = 0.0
x3 = 45.0
x4 = 0.0
y1 = 0.0
y3 = 1.0


,name,shadow price,slack
0,manpower,-0.0,5.0
1,material_A,-0.0,55.0
2,material_B,20.0,-0.0
3,x1_constraint,20.0,-0.0
4,x3_constraint,-0.0,55.0
5,y_constraint,-0.0,-0.0


In [204]:
model = LpProblem('Feed_mix_problem', LpMinimize)
x = {i: LpVariable(f"x{i}", 0) for i in range(1, 4)}
model += 41*x[1] + 36*x[2] + 96*x[3], 'objective'

model += 20*x[1] + 30*x[2] + 70*x[3] >= 110
model += 10*x[1] + 10*x[2] >= 18
model += 50*x[1] + 30*x[2] >= 90
model += 6*x[1] + 2.5*x[2] + 10*x[3] >= 14

model.solve()

print('Status', LpStatus[model.status])
print('Ojective', model.objective.value())
for v in model.variables():
    print(v, '=', v.varValue)

Status Optimal
Ojective 148.61089482
x1 = 0.59533074
x2 = 2.0077821
x3 = 0.54085603


In [226]:
model = LpProblem('Blending_Problem', LpMaximize)
x = {i: LpVariable(f"x{i}", 0) for i in range(1, 5)}
y = {i: LpVariable(f"y{i}", 0) for i in range(1, 5)}
model += 19.5*lpSum(x) + 22*lpSum(y) - (17*(x[1]+y[1]) + 18*(x[2]+y[2]) + 20.5*(x[3]+y[3]) + 23*(x[4]+y[4]))

model += (86*x[1] + 88*x[2] + 92*x[3] + 96*x[4]) >= 89 * lpSum(x)
model += (86*y[1] + 88*y[2] + 92*y[3] + 96*y[4]) >= 93 * lpSum(y)
model += x[1] + y[1] <= 20000
model += x[2] + y[2] <= 15000
model += x[3] + y[3] <= 15000
model += x[4] + y[4] <= 10000
model += lpSum(x) <= 35000
model += lpSum(y) <= 23000

model.solve()

print('Status', LpStatus[model.status])
print('Objective', model.objective.value())
for v in model.variables():
    print(v, '=', v.varValue)

Status Optimal
Objective 42142.855684999995
x1 = 11428.571
x2 = 9107.1429
x3 = 14464.286
x4 = 0.0
y1 = 0.0
y2 = 5892.8571
y3 = 535.71429
y4 = 10000.0


In [4]:
model = LpProblem('Inventory_Problem', LpMinimize)
r = {i: LpVariable(f"r{i}", lowBound=0, upBound=420) for i in range(1, 7)}
o = {i: LpVariable(f"o{i}", lowBound=0, upBound=80) for i in range(1, 7)}
i = {i: LpVariable(f"i{i}", 0) for i in range(1, 7)}
model += 40*lpSum(r) + 45*lpSum(o) + 3*lpSum(i) , 'Objective'

model += r[1] + o[1] == 360 + i[1]
model += r[2] + o[2] + i[1] == 430 + i[2]
model += r[3] + o[3] + i[2] == 380 + i[3]
model += r[4] + o[4] + i[3] == 450 + i[4]
model += r[5] + o[5] + i[4] == 520 + i[5]
model += r[6] + o[6] + i[5] == 440 + i[6]

model.solve()

print('Status', LpStatus[model.status])
print('Objective', model.objective.value())
pd.DataFrame([{'var':v, 'value':v.varValue} for v in model.variables()])



Status Optimal
Objective 103960.0


,var,value
0,i1,10.0
1,i2,0.0
2,i3,40.0
3,i4,20.0
4,i5,0.0
5,i6,0.0
6,o1,0.0
7,o2,0.0
8,o3,0.0
9,o4,10.0
